## TK PROJECT TITLE 
Research Computing for Earth Sciences Final Project 
December 2020
Miriam Nielsen

### Research Question
TK



In [3]:
# Needed Packages
import cartopy.crs as ccrs
import cartopy
import numpy as np
import pandas as pd
import xarray as xr

from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)

from IPython import display
display.set_matplotlib_formats('retina')

### Data Loading

TK DATA USED

### TK SOME ANALYSIS

### Discussion

**TK**

TK

**TK**

TK